Tarefa 65
Sistema de identificación de entrada.

Utilizando o aprendido coa detección de caras, trátase de crear un pequeno sistema de control de acceso a un recinto deportivo. O sistema debe detectar cando aparece na cámara unha cara e tentar identicala. Se a cara é coñecida indicará a hora a que entra ese usuario e almacenará a hora de entrada. Unha vez se deixa de detectar a cara dáselle un pequeno marxe de tempo, e se esa mesma cara volve a aparecer considerarase que o usuario abandona o recinto e se indicará o tempo que pasou dentro.

Se a cara non é coñecida dará unha alerta.

As caras deben cargarse dalgún sistema tipo BD, polo que previamente deberás cargar os datos necesarios para o recoñecemento facial nese almacén.

O código de CV para capturar imaxes da cámara sería algo así:



import cv2

video_capture = cv2.VideoCapture(0)


while True:
    # Captura un frame
    ret, frame = video_capture.read()

    # para procesar o frame con outras ferramentas seguramente habería que convertilo a RGB 
    # (Lembra que opencv traballa con BGR)

    # Amosa a imaxe resultante
    cv2.imshow('Video', frame)

    # Pulsa 'q' para saír!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera a cámara
video_capture.release()
cv2.destroyAllWindows()  
  



In [3]:
import cv2
import dlib
import pandas as pd
import numpy as np
from datetime import datetime

# Función para calcular la distancia euclidiana entre dos vectores
def euclidean_distance(vector1, vector2):
    return np.linalg.norm(vector1 - vector2)

# Cargar la base de datos de caras
database = pd.read_csv('proba.csv')

# Convertir las características de las caras en arrays numpy
database['fingerprint'] = database['fingerprint'].apply(lambda x: np.array([float(i) for i in x.split(',')]))

# Inicializar el detector de caras de DLIB
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Inicializar la captura de video desde la cámara
video_capture = cv2.VideoCapture(0)

# Inicializar variables para el control de tiempo
last_seen = datetime.now()
current_user = None

# Bucle principal
while True:
    # Captura un frame de la cámara
    ret, frame = video_capture.read()

    # Convertir el frame a escala de grises
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar caras en el frame
    faces = detector(gray_frame)

    # Para cada cara detectada
    for face in faces:
        # Obtener las coordenadas de la cara
        x, y, w, h = face.left(), face.top(), face.width(), face.height()

        # Extraer las características de la cara
        shape = predictor(gray_frame, face)
        face_descriptor = np.array([shape.part(i).x for i in range(68)] + [shape.part(i).y for i in range(68)])

        # Comparar la cara detectada con las caras en la base de datos
        for index, row in database.iterrows():
            distance = euclidean_distance(face_descriptor, row['fingerprint'])
            if distance < umbral_de_identificacion:  # Ajustar el umbral según sea necesario
                current_user = row['nome']
                if current_user is not None and current_user != row['nome']:
                    # Si la cara detectada pertenece a un usuario diferente al anteriormente identificado
                    # Registra la hora de entrada y actualiza el usuario actual
                    entry_time = datetime.now()
                    print(f"¡Bienvenido, {current_user}! Hora de entrada: {entry_time}")
                    current_user = row['nome']
                    last_seen = entry_time

        # Dibujar un rectángulo alrededor de la cara
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Actualizar el estado del usuario si no se detecta una cara en un tiempo determinado
    if current_user is not None and (datetime.now() - last_seen).total_seconds() > tiempo_de_espera:
        exit_time = datetime.now()
        print(f"¡Adiós, {current_user}! Tiempo de salida: {exit_time}. Duración de la visita: {exit_time - last_seen}")
        current_user = None

    # Mostrar el frame con las caras detectadas
    cv2.imshow('Video', frame)

    # Salir del bucle si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar todas las ventanas
video_capture.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: 'proba.csv'